In [1]:
import os

cur_notebook_dir = os.getcwd()
os.chdir(os.path.join(os.getcwd(), '../'))
base_dir = os.getcwd()
key_dir = os.path.join(base_dir, 'keys')
data_dir = os.path.join(base_dir, 'data')

os.chdir(base_dir)
print(os.getcwd())

from dotenv import load_dotenv
print(load_dotenv(dotenv_path= os.path.join(key_dir, ".env")))
os.chdir(cur_notebook_dir)

c:\Users\thdwo\Documents\Github
True


In [2]:
os.listdir(data_dir + '/pdf')

['엘루이 피자, 가마로빚은고등어.pdf', '오키나와 렌트카.pdf']

## 스크립트 가져오기
- 네이버 블로그에서 관리자 페이지에서 다운로드 받은 PDF 파일에서 스크립트 
- PyMuPDF 사용

In [4]:
import pymupdf
doc = pymupdf.open(os.path.join(data_dir, 'pdf/엘루이 피자, 가마로빚은고등어.pdf'))
base_url = "http://blog.naver.com/ddunidubab"
for i, page in enumerate(doc):
    text = page.get_text()
    if text 
    print(i, text)

    # image_list = page.get_images()
    # print(image_list)

SyntaxError: expected ':' (3551223551.py, line 6)

In [5]:
# from PIL import Image
# img_path = os.path.join(data_dir, 'images/whole_test.png')
# img = Image.open(img_path)
# img

## PyTesseract

In [18]:
# import pytesseract as pts
# import cv2

# img_path
# img_tmp = cv2.imread(img_path)

# text = pts.image_to_string(img_tmp, lang = 'kor')
# print(text)

좋았다

1

핀 - 본

제 ㅜ 을     뼈

모 비 해     인 때
가 뷔페     기 =
메 에 호     어 『
때 라 종

자미 회

고 원 불

더 홈 권

개

메 알포

부역

<

2차로

위치

호프

|  크라운

서울 송파구 송파대로49:



## EasyOCR

In [5]:
from easyocr import Reader
import numpy as np

reader = Reader(['ko'], gpu = False)

result = reader.readtext(np.array(img))

Using CPU. Note: This module is much faster with a GPU.


In [6]:
res_text = ''.join(s[1] + '\n' for s in result)
print(res_text)

대학원 진구들과 수업 끝나고 헬림할 곁
다녀올던 남양주
친구가 맛있고 분위기 앞는 피자집이
있다고 해서 데려가주다 =국국
맛이따!
여 7 초야 !
위치 / 주차
경기 남양주시 와부움 팔당로1 39번길 79-29
한혹에서 [;723 온고재
경기도 남양주시 와부음 팔당로 1 39번길 19-29 온고재



In [18]:
import base64
import re

def encode_image(img_path):
    """
    Endoces an image to base64 string    
    """
    with open(img_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def extract_code(text):
    pattern = r'```python\s+(.*?)\s+```'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    return None

In [19]:
from openai import OpenAI

client = OpenAI()
img_obj = encode_image(img_path)

response = client.chat.completions.create(
    model = 'gpt-4-vision-preview',
    messages = [
        {
            'role': 'user',
            'content' : [
                {
                    'type': 'text',
                    'text': f'''
                    The input image is a screen capture of a blog post. It is written in Korean.
                    Your job is to crop the input image into an image that only contains the main content.
                    The OUTPUT should be a set of coordinates for the output cropped image. 4 pairs of x and y coordinates.
                    Here are some characteristics of the main content part.
                    - The background color of main content section is white.
                    - The main content section starts under the main title of the post.
                    - The end of OUTPUT post is located in above the tags.

                    *OUTPUT
                    '''
                },
                {
                    'type': 'image_url',
                    'image_url': {
                        'url' : f'data:image/png;base64,{img_obj}'
                    }
                }
            ]
        }
    ]
)

BadRequestError: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'image_too_large'}}

In [ ]:
print(response.to_dict()['choices'][0]['message']['content'])

In [12]:
from openai import OpenAI

client = OpenAI()
img_obj = encode_image(img_path)

response = client.chat.completions.create(
    model = 'gpt-4-vision-preview',
    messages = [
        {
            'role': 'user',
            'content' : [
                {
                    'type': 'text',
                    'text': f'''
                    이 이미지는 한국어로 된 블로그 포스트의 일부분이야.
                    포스트 내에 있는 텍스트로 된 본문은 아래 스크립트와 같아.
                    스크립트를 참조해서 포스트와 같은 모양으로 출력되게 마크다운 형식으로 출력해줘.
                    라인은 포스트내 line break로 구분된 한 줄을 의미해.
                
                    - 스크립트는 맞춤법이 틀린 경우 교정해줘.
                    - 라인이 Heading으로 보이는 경우 적당히 사이즈가 맞는 레벨의 Heading을 적용해줘.
                    - 라인이 heading이 아닌 경우 출력에서 각 라인 사이에 <center>라인 내용</center>의 형식으로 태그를 넣어줘
                        예를 들어 라인의 내용이 "우연히 발견한 가게이다!" 라면 "<center>우연히 발견한 가게이다!</center>"로 나와야 해
                    - 라인안에 이모지가 있는 경우 비슷한 이모지를 넣어줘.
                    - 포스트 내 있는 이미지는 마크다운 본문 내 ((이미지 내용)) 으로 처리해서 작성해줘.
                        - ((이미지 내용))안에는 이미지가 무엇인지 짧게 묘사해줘.
                        - OCR이 스크립트를 처리할 때 이미지 안에 있는 글자를 인식했을 수도 있어. 이 경우로 보인다면 스크립트의 내용은 무시해줘.
                    - 포스트 중간에 광고가 나오는 영역이 있는데 이 부분은 무시해줘.
                        - 회색 선으로 감싸져 있고 우측 상단에 "AD X"라고 표시돼있어.

                    *스크립트
                    {res_text}

                    *출력
                    '''
                },
                {
                    'type': 'image_url',
                    'image_url': {
                        'url' : f'data:image/png;base64,{img_obj}'
                    }
                }
            ]
        }
    ]
)

In [13]:
print(response.to_dict()['choices'][0]['message']['content'])

```markdown
## 대학원 진구들과 수업 끝나고 함께한 점심
<center>다녀온 남양주</center>
<center>친구가 맛있고 분위기 있는 피자집이</center>
<center>있다고 해서 데려가줬다 🍕</center>
<center>맛있다!</center>
<center>여기 추천! 🐑</center>

((이미지 내용: 만화 소재의 감탄을 나타내는 캐릭터와 함께 "맛있다!" "여기 추천!" 이라는 글귀가 쓰여있음))

## 위치 / 주차
<center>경기 남양주시 와부읍 팔당로 139번길 19-29</center>
<center>한옥에서 피자를 먹을 수 있는</center>
<center>경기도 남양주시 와부읍 팔당로 139번길 19-29 온고재</center>
```
Note: I have corrected the spelling and spacing in the Korean text provided in the script, and I have applied markdown syntax. The image has been described within double parentheses as per the instruction, and the image description does not include any OCR text that might have been mistakenly recognized from the image. Additionally, the requested emoji has been included in the text.

